In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from hyperloglog import HyperLogLog
from nltk import ngrams
from scipy.stats import skew, kurtosis
import urllib.parse


import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output, State
import os
from dabl import detect_types
from hyperloglog import HyperLogLog
from collections import namedtuple, Counter as count
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from hyperloglog import HyperLogLog
from pyod.models.knn import KNN
from abc import abstractmethod
from dabl import detect_types
from nltk.util import ngrams
from pathlib import Path
from enum import IntEnum
import pandas as pd
import numpy as np
import copy
import time
import math
import plotly.express as px
from urllib.parse import unquote
from dash import Dash, dcc, html, Input, Output, State, callback_context



In [ ]:
#data loader.py 
import os
import pandas as pd
import json
import numpy as np
import os
import pandas as pdaoao
import numpy as np
import json
from collections import namedtuple, Counter as count

root_directory_path = 'C:/Users/Isha/ERP/DataRepo2/'

def populate_directory_options(directory_path):
    options = []
    for root, dirs, files in os.walk(directory_path):
        for dir_name in dirs:
            options.append({'label': os.path.join(root, dir_name), 'value': os.path.join(root, dir_name)})
    return options

def detect_header(file_path, delimiter=','):
    try:
        df = pd.read_csv(file_path, delimiter=delimiter, nrows=2)
        if df.empty:
            return False
        first_row = df.iloc[0]
        second_row = df.iloc[1]
        if first_row.dtype == object and second_row.dtype != object:
            return True
        return False
    except Exception as e:
#         print(f"Failed to read {file_path} to detect header: {e}")
        return False

def DetectHeader(df):
    if df is None:
            return False
    first_row = df.iloc[0]
    second_row = df.iloc[1]
    if first_row.dtype == object and second_row.dtype != object:
        return True
    return False

def try_read_csv(file_path, delimiter=','):
    try:
        if detect_header(file_path, delimiter):
            return pd.read_csv(file_path, delimiter=delimiter)
        else:
            return pd.read_csv(file_path, delimiter=delimiter, header=None)
    except Exception as e:
#         print(f"Failed to read {file_path} as CSV with delimiter '{delimiter}': {e}")
        return None

def try_read_json_lines(file_path):
    try:
        return pd.read_json(file_path, lines=True)
    except Exception as e:
#         print(f"Failed to read {file_path} as JSON Lines: {e}")
        return None

def try_read_text_as_json(file_path):
    try:
        records = []
        with open(file_path, 'r') as f:
            for line in f:
                records.append(json.loads(line.strip()))
        return pd.DataFrame(records)
    except Exception as e:
#         print(f"Failed to read {file_path} as JSON per line: {e}")
        return None

def try_read_text_generic(file_path):
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
        delimiters = [',', '\t', ' ']
        for delimiter in delimiters:
            try:
                has_header = detect_header(file_path, delimiter)
                if has_header:
                    df = pd.read_csv(file_path, delimiter=delimiter)
                else:
                    df = pd.read_csv(file_path, delimiter=delimiter, header=None)
                return df
            except Exception as e:
#                 print(f"Failed to parse {file_path} with delimiter '{delimiter}': {e}")
                return None
    except Exception as e:
#         print(f"Failed to read {file_path} as generic text: {e}")
        return None

def load_files(root_directory_path):
    dataframes_dict = {}
    for dirpath, dirnames, filenames in os.walk(root_directory_path):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            df = None
            if file.endswith('.csv'):
                df = pd.read_csv(file_path)
            elif file.endswith('.xlsx') or file.endswith('.xls'):
                df = pd.read_excel(file_path)
            elif file.endswith('.json'):
                df = pd.read_json(file_path)
            elif file.endswith('.jsonl'):
                df = pd.read_json(file_path, lines=True)
            elif file.endswith('.data'):
                df = pd.read_csv(file_path, delimiter=",")
            else:
                df = try_read_text_as_json(file_path)
                if df is None:
                    df = try_read_csv(file_path, delimiter=',')
                if df is None:
                    df = try_read_csv(file_path, delimiter='\t')
                if df is None:
                    df = try_read_text_generic(file_path)
            if df is not None:
                dataframes_dict[file_path] = df
    return dataframes_dict


def load_single_file(file_path):
    df = None
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            df = pd.read_excel(file_path)
        elif file_path.endswith('.json'):
            df = pd.read_json(file_path)
        elif file_path.endswith('.jsonl'):
            df = pd.read_json(file_path, lines=True)
        elif file_path.endswith('.data'):
            df = pd.read_csv(file_path, delimiter=",")
        elif file_path.endswith('.txt'):
            df = try_read_text_as_json(file_path)
            if df is None:
                df = try_read_csv(file_path, delimiter=',')
            if df is None:
                df = try_read_csv(file_path, delimiter='\t')
            if df is None:
                df = try_read_text_generic(file_path)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
    print(df)
    return df

def dataType_distribution(df):
    data_types = df.dtypes.value_counts()
    return data_types.to_json()


def overall_outlier_percentage(df):
    numeric_cols = df.select_dtypes(include=[np.number])
    if numeric_cols.empty:
        return 0.0
    outliers = ((numeric_cols < (numeric_cols.quantile(0.25) - 1.5 * (numeric_cols.quantile(0.75) - numeric_cols.quantile(0.25)))) |
                (numeric_cols > (numeric_cols.quantile(0.75) + 1.5 * (numeric_cols.quantile(0.75) - numeric_cols.quantile(0.25))))).sum()
    total_values = numeric_cols.count()
    outlier_percent = (outliers.sum() / total_values.sum()) * 100
    return outlier_percent

def calculate_data_quality_dynamic(completeness_score, outlier_percentage, uniqueness_score, rows, columns):
    # Dynamic weight adjustment based on shape
    if rows > 10000 or columns > 100:  # Large datasets
        completeness_weight = 0.3
        outlier_weight = 0.4
        uniqueness_weight = 0.2
        shape_weight = 0.1
    elif rows < 100 and columns < 10:  # Small datasets
        completeness_weight = 0.4
        outlier_weight = 0.2
        uniqueness_weight = 0.3
        shape_weight = 0.1
    else:  # Medium datasets
        completeness_weight = 0.3
        outlier_weight = 0.3
        uniqueness_weight = 0.3
        shape_weight = 0.1
    # Define thresholds for optimal, too sparse, and too complex
    optimal_row_lower = 100
    optimal_row_upper = 10000
    optimal_column_lower = 5
    optimal_column_upper = 50
    
    # Calculate shape bonus/penalty
    shape_bonus = 0
    
    # Bonus for optimal number of rows and columns
    if optimal_row_lower <= rows <= optimal_row_upper and optimal_column_lower <= columns <= optimal_column_upper:
        shape_bonus += 10  # Give bonus points for optimal shape
    else:
        # Penalty for too sparse or too complex datasets
        if rows < optimal_row_lower or columns < optimal_column_lower:
            shape_bonus -= 10  # Penalty for too sparse datasets
        if rows > optimal_row_upper or columns > optimal_column_upper:
            shape_bonus -= 10  # Penalty for too complex datasets

    
    # Calculate data quality percent with dynamic weighting
    data_quality = (completeness_score * completeness_weight +
                    (100 - outlier_percentage) * outlier_weight +
                    uniqueness_score * uniqueness_weight +
                    shape_bonus*shape_weight)
    data_quality_percentage = max(0, min(100, data_quality))

    return data_quality_percentage #score

# Check for columns containing lists and convert them to tuples
def convert_lists_to_tuples(row):
    return [tuple(item) if isinstance(item, list) else item for item in row]



def analyze_files(root_directory_path):
    dataframes_dict = load_files(root_directory_path)
#     print(dataframes_dict)
    data = []

    for file_path, df in dataframes_dict.items():
        file_name = os.path.basename(file_path)
        size_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
        rows, columns = df.shape
#         print(df)
        completeness_score = (1 - df.isnull().mean().mean()) * 100
        outlier_percentage = overall_outlier_percentage(df)
        # Apply the conversion to the entire DataFrame
        df_converted = df.apply(convert_lists_to_tuples, axis=1)
        uniqueness_score = (df_converted.drop_duplicates().shape[0] / rows) * 100
        data_type_distribution = dataType_distribution(df)
        data_quality_percent = calculate_data_quality_dynamic(completeness_score, outlier_percentage, uniqueness_score, rows, columns)

        data.append({
            'name': file_name,
            'size (MB)': size_mb,
            'rows': rows,
            'columns': columns,
            'completeness_score': completeness_score,
            'outlier_percentage': outlier_percentage,
            'uniqueness_score': uniqueness_score,
            'data_type_distribution': data_type_distribution,
            'data_quality_percent': data_quality_percent,
            'file_path': file_path  # Store file path for later use
        })

    df_summary = pd.DataFrame(data)
    print("Columns in DataFrame:", df_summary.columns)
#     print("Data in DataFrame:", df_summary.head())
    df_summary['volume'] = df_summary['rows'].astype(str) + ' x ' + df_summary['columns'].astype(str)
    df_summary = df_summary.sort_values('data_quality_percent', ascending=False).reset_index(drop=True)
    df_summary['ranking'] = df_summary.index + 1
    # Extract file extensions from the file names and add a new column 'extension'
    df_summary['extension'] = df_summary['name'].apply(lambda x: os.path.splitext(x)[1].lstrip('.').lower())

#     print(profile)
    return df_summary
#     return dataframes_dict


In [ ]:
#data profiler 
import pandas as pd
import numpy as np

class DataProfiler:
    def __init__(self, df, column):
        self.df = df  # The DataFrame containing the data
        self.column = column  # The specific column to profile

    def completeness_score(self):
        completeness = self.df[self.column].notnull().sum() / self.df[self.column].shape[0] * 100
        return completeness

    def uniqueness_score(self):
        uniqueness = self.df[self.column].nunique() / self.df[self.column].count() * 100
        return uniqueness

    def data_quality_percent(self):
        data_quality = 100 - self.total_error_percent()
        return data_quality
    
    def get_outliers(self,df, column):
#         print("entered nested func")
        if pd.api.types.is_numeric_dtype(df[column]): 
        # IQR method for outlier detection
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            is_outlier = (df[column] < lower_bound) | (df[column] > upper_bound)
            outliers = df[is_outlier].copy()
            outliers['Row No. (index)'] = outliers.index
            outliers['Actual Value'] = outliers[column]
            outliers['Expected Value (Mean)'] = df[column].mean()
            return outliers.reset_index(drop=True)[['Row No. (index)', 'Actual Value', 'Expected Value (Mean)']]
        return pd.DataFrame()
    
    
    def get_rare_values(self,df, column, threshold):
        # Convert lists to tuples for hashable operation, if applicable
        df[column] = df[column].apply(lambda x: tuple(x) if isinstance(x, list) else x)
        # Calculate the frequency of each value in the column
        value_counts = df[column].value_counts(normalize=True)
#         print("HELLLOOOOOOOO freq 1")

        # Identify values that occur less frequently than the threshold
        rare_values = value_counts[value_counts < threshold].index
#         print("HELLLOOOOOOOO freq 2")
#         print(rare_values)

        # Return the DataFrame of outliers
        outliers = df[df[column].isin(rare_values)].copy()
        outliers['Row No. (index)'] = outliers.index
        outliers['Actual Value'] = outliers[column]
        mode_series = df[column].mode()
        if not mode_series.empty:
            outliers['Expected Value (Mode)'] = mode_series.iloc[0]
        else:
            outliers['Expected Value (Mode)'] = None  # or some default value            
        return outliers.reset_index(drop=True)[['Row No. (index)', 'Actual Value', 'Expected Value (Mode)']]

    def detect_outliers_iqr(self, df, column):
#         print("entered IQR")
        df_extended, extended_columns = extend_dataframe_with_rules(df, column)
        outlier_list = []
        
        nan_columns = df_extended.columns[df_extended.isna().all()].tolist()
        df_extended = df_extended.dropna(how='all', axis=1)  # Drop extended columns where all values are NaN
        extended_columns = [col for col in extended_columns if col not in nan_columns]  # Update extended_columns
        
        # Detect outliers in original column
#         print("right before enterng nested func")
        outliers_original = self.get_outliers(df, column)
#         print("HELLLOOOOOOOO IQR")
#         print(outliers_orignial)
        outlier_list.append(outliers_original)

        # Detect outliers in extended columns
        for col in extended_columns:
            if col in df_extended.columns and pd.api.types.is_numeric_dtype(df_extended[col]):
                outliers_extended = self.get_outliers(df_extended, col)
                outlier_list.append(outliers_extended)

        # Concatenate all outliers
        if outlier_list:
            outlier_df = pd.concat(outlier_list, axis=0).drop_duplicates()
            return outlier_df
        return pd.DataFrame()
    
    def detect_outliers_frequency(self, df, column, threshold=0.05):
#         print("entered FREQ")
        df_extended, extended_columns = extend_dataframe_with_rules(df, column)
        outlier_list = []

        nan_columns = df_extended.columns[df_extended.isna().all()].tolist()
        df_extended = df_extended.dropna(how='all', axis=1)  # Drop extended columns where all values are NaN
        extended_columns = [col for col in extended_columns if col not in nan_columns]  # Update extended_columns
        
        
        # Detect outliers in the original column
        outliers_original = self.get_rare_values(df, column, threshold)
        outlier_list.append(outliers_original)
#         print("outlier list right now - ",outlier_list)
        # Detect outliers in extended columns
        for col in extended_columns:
            if col in df_extended.columns:
#                 print(col)
                outliers_extended = self.get_rare_values(df_extended, col, threshold)
                outlier_list.append(outliers_extended)

        # Concatenate all outliers found
        if outlier_list:
            outlier_df = pd.concat(outlier_list, axis=0).drop_duplicates()
            return outlier_df

        # Return an empty DataFrame if no outliers are found
        return pd.DataFrame()
    
    def isIndex(self):
        # Check if column exists in DataFrame
        if self.column not in self.df.columns:
            raise ValueError(f"Column '{self.column}' does not exist in the DataFrame.")

        # Check for uniqueness
        is_unique = self.df[self.column].is_unique

        # Check for missing values
        has_no_missing = self.df[self.column].notna().all()

        # Combine both conditions
        return is_unique and has_no_missing
    
    def mean_median_difference(self):
        if pd.api.types.is_numeric_dtype(self.df[self.column]):
            mean_val = self.df[self.column].mean()
            median_val = self.df[self.column].median()
            difference = abs(mean_val - median_val)
            return difference
        return "N/A"

    def generate_profile(self):
        return {
            'completeness_score': self.completeness_score(),
            'outlier_percentage': 100,
            'uniqueness_score': self.uniqueness_score(),
            'data_quality_percent': self.data_quality_percent(),
            'isIndex' : self.isIndex(),
            'Mean/MedianImbalance' : mean_median_difference()
            
        }


In [ ]:
#error detection.py
import re
import unicodedata
import email.utils
from collections import defaultdict
import inspect
import time

rules = defaultdict(list)

def rule(rule):
    spec = inspect.getfullargspec(rule)

    if len(spec.args) != 1:
        raise ValueError("Invalid rule {}".format(rule.__name__))

    input_type = spec.annotations[spec.args[0]]
    rules[input_type].append(rule)
    return rule

def apply_rules(df, column):
    data_type = df[column].dtype
    results = []
    extended_columns = []
    
    if pd.api.types.is_string_dtype(data_type):
        values = df[column].dropna()
        for value in values:
            result = {}
            for rule in rules.get(str, []):
                rule_result = rule(value)
                result[rule.__name__] = rule_result
                for idx, res in enumerate(rule_result):
                    extended_columns.append(f"{column}_{rule.__name__}_{idx}")
            results.append(result)
    
    elif pd.api.types.is_numeric_dtype(data_type):
        values = df[column].dropna()
        for value in values:
            result = {}
            for rule in rules.get(float, []):
                rule_result = rule(value)
                result[rule.__name__] = rule_result
                for idx, res in enumerate(rule_result):
                    extended_columns.append(f"{column}_{rule.__name__}_{idx}")
            results.append(result)
    
    return results, extended_columns

def extend_dataframe_with_rules(df, column):
    rule_results, extended_columns = apply_rules(df, column)
    
    # Convert list of dictionaries to DataFrame
    rule_df = pd.DataFrame(rule_results)
    
    # Convert boolean columns to numeric
    for col in rule_df.columns:
        if rule_df[col].dtype == 'bool':
            rule_df[col] = rule_df[col].astype(int)
    
    # Create extended columns with default NaN values
    for ext_col in extended_columns:
        if ext_col not in rule_df.columns:
            rule_df[ext_col] = pd.NA
    
    # Merge with original DataFrame
    df_extended = pd.concat([df, rule_df], axis=1)
    return df_extended, extended_columns


@rule
def string_case(s: str) -> ("upper case", "lower case", "title case"):
    return (s.isupper(), s.islower(), s.istitle())

@rule
def string_is_digit(s: str) -> ("is digit",):
    return (s.isdigit(),)

@rule
def length(s: str) -> ("length",):
    return (len(s),)

@rule
def signature(s: str) -> ("signature",):
    return (",".join(map(unicodedata.category, s)),)

NUMBERS = re.compile(r"(^s)?\d+")

@rule
def strp(s: str) -> ("strp",):
    return (NUMBERS.sub("<num>", s),)

HTML5_EMAIL_VALIDATOR = re.compile(r"^[a-zA-Z0-9.!#$%&'*+/=?^_`{|}~-]+@[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?(?:\.(?P<ext>[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?))*$")

@rule
def email_checks(s: str) -> ("simple email check",):
    return (HTML5_EMAIL_VALIDATOR.match(s) != None,)

@rule
def email_domain(s: str) -> ("email domain",):
    match = HTML5_EMAIL_VALIDATOR.match(s)
    return (match.group("ext").lower() if match else "NONE",)

@rule
def id(s: str) -> ("id",):
    return (s,)

@rule
def empty(s: str) -> ("empty",):
    return (s == "" or s.isspace(),)

@rule
def int_id(x: int) -> ("id",):
    return (x,)

@rule
def int_kill(x: int) -> ("nil",):
    return (None,)

@rule
def float_id(f: float) -> ("id",):
    return (f,)

def _bits(*positions):
    def bits(i: int) -> tuple("bit {}".format(pos) for pos in positions):
        return ((i >> pos) & 1 for pos in positions)
    return bits

def _mod(*mods):
    def mod(i: int) -> tuple("mod {}".format(mod) for mod in mods):
        return (i % mod for mod in mods)
    return mod

def _div(*mods):
    def div(i: int) -> tuple("div {}".format(mod) for mod in mods):
        return (i % mod == 0 for mod in mods)
    return div

DATE_PROPS = "tm_year", "tm_mon", "tm_mday", "tm_hour", "tm_min", "tm_sec", "tm_wday", "tm_yday"

@rule
def unix2date(timestamp: int) -> DATE_PROPS:
    t = time.gmtime(timestamp)
    return map(lambda a: getattr(t, a), DATE_PROPS)

@rule
def unix2date_float(timestamp: float) -> DATE_PROPS:
    return unix2date(int(timestamp))

@rule
def fracpart(x: float) -> ("frac part",):
    return (x - int(x),)

@rule
def is_weekend(timestamp: int) -> ("is weekend",):
    wday = time.gmtime(timestamp).tm_wday
    wkend = wday in [5, 6]
    return (wkend,)

rule(_bits(0, 1, 2, 3, 4, 5))
rule(_div(3, 5))
rule(_mod(10))


In [ ]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from dash.dependencies import Input, Output, State
from dash import dash_table
import os
import math
import json
# from data_loader import load_files, analyze_files, load_file
# from data_profiler import DataProfiler
# from error_detection import apply_rules, extend_dataframe_with_rules

# Load the datasets
root_directory_path = 'C:/Users/Isha/ERP/DataRepo1/'
df_summary = analyze_files(root_directory_path)

# Extract the folder name dynamically
folder_name = os.path.basename(os.path.normpath(root_directory_path))

# Initialize the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP],suppress_callback_exceptions=True)
server = app.server

# Layout of the main app
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Index page layout
# Index page layout
index_page = html.Div([
    dbc.Row(
        dbc.Col(
            html.H1("Data Quality Dashboard", style={'width': '100%', 'whiteSpace': 'normal'}),
            width={'size': 6, 'offset': 3}
        ),
    ),
    dbc.Row(
        dbc.Col(html.Hr(), width={'size': 8, 'offset': 2}),
    ),
    dbc.Row(
        dbc.Col(
            html.Div([
                html.H4("Data Repo Summary", style={'width': '100%', 'whiteSpace': 'normal'}),
            ], style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'width': '100%'}),
        )
    ),
    dbc.Row(
        dbc.Col(
            html.Div([
                html.Div(f"Name: {folder_name}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'whiteSpace': 'normal'}),
                html.Div(f"Size: {df_summary['size (MB)'].sum()} MB", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'whiteSpace': 'normal'}),
                html.Div(f"Number of Files: {len(df_summary)}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'whiteSpace': 'normal'}),
                html.Div(f"Path: {root_directory_path}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'whiteSpace': 'normal'}),
                html.Div(f"Min File Size: {df_summary['size (MB)'].min()} MB", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'whiteSpace': 'normal'}),
                html.Div(f"Max File Size: {df_summary['size (MB)'].max()} MB", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'whiteSpace': 'normal'})
            ], style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'display': 'flex', 'flexDirection': 'row', 'width': '100%'}),
        )
    ),
    dbc.Row([
        dbc.Col([
            html.Label('Sort by:', style={'width': '100%', 'flex': '1'}),
            dcc.Dropdown(
                id='sort_by',
                options=[
                    {'label': 'Name', 'value': 'name'},
                    {'label': 'Size', 'value': 'size (MB)'},
                    {'label': 'Rows', 'value': 'rows'},
                    {'label': 'Columns', 'value': 'columns'},
                    {'label': 'Completeness Score', 'value': 'completeness_score'},
                    {'label': 'Outlier Percentage', 'value': 'outlier_percentage'},
                    {'label': 'Uniqueness Score', 'value': 'uniqueness_score'},
                    {'label': 'Data Type Distribution', 'value': 'data_type_distribution'},
                    {'label': 'Data Quality Percent', 'value': 'data_quality_percent'},
                    {'label': 'Ranking', 'value': 'ranking'}
                ],
                value='data_quality_percent',
                style = {'width' : '100%'}
            ),
        ], width=4,style={'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),
        dbc.Col([
            html.Label('File Type filter:', style={'width': '100%', 'whiteSpace': 'normal'}),
            dcc.Dropdown(
                id='file_type_filter',
                options=[{'label': ext, 'value': ext} for ext in df_summary['extension'].unique()],
                value=None,
                multi=True
            )
        ], width=4, style={'width': '100%', 'flex': '1'})
    ], style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px', 'display': 'flex', 'flexDirection': 'row', 'flexWrap': 'wrap'}),
    dbc.Row(
        dbc.Col(
            dash_table.DataTable(
                id='data_table',
                columns=[
                    {'name': 'Name', 'id': 'name'},
                    {'name': 'Size (MB)', 'id': 'size (MB)'},
                    {'name': 'Volume', 'id': 'volume'},
                    {'name': 'Completeness Score', 'id': 'completeness_score'},
                    {'name': 'Outlier Percentage', 'id': 'outlier_percentage'},
                    {'name': 'Uniqueness Score', 'id': 'uniqueness_score'},
                    {'name': 'Data Type Distribution', 'id': 'data_type_distribution'},
                    {'name': 'Data Quality Score', 'id': 'data_quality_percent'},
                    {'name': 'Ranking', 'id': 'ranking'}
                ],
                data=df_summary.to_dict('records'),
                filter_action="native",
                sort_action="native",
                row_selectable="single",
                selected_rows=[],
                page_action="native",
                page_size=10,
                style_table={'overflowX': 'auto', 'overflowY': 'auto', 'maxHeight': '500px'},  
                style_cell={
                    'height': 'auto',
                    'minWidth': '100px', 'width': 'auto', 'maxWidth': '500px',
                    'whiteSpace': 'normal',  # Enables text wrapping
                    'textAlign': 'left',
                    'overflow': 'hidden',
                    'textOverflow': 'ellipsis'
                }
            ), width={'size': 10, 'offset': 1}
        ),
    ),
    dbc.Row([
        dbc.Col(
            dbc.Button("View Details", id='view-details-btn', color='primary', disabled=True, href='/details'),
            width={'size': 2, 'offset': 5}
        ),
        dbc.Col(
            dbc.Button("Back to Index", id='back-to-index-btn', color='secondary', href='/'),
            style={'display': 'none'}
        )]
    )
])


# Detail page layout
detail_page = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Div([
                # Title for the navigation section
                html.H6("Attributes", className="display-2", style={'padding': '10px 0', 'textAlign': 'center','fontSize': '24px','fontWeight': 'bold',  }),
                dbc.Nav(id='attribute-nav', vertical=True, pills=True, style={'overflowY':'auto'}),
        ])], width=2, className="bg-light",style={'overflowY':'auto'}),
        dbc.Col([
            html.Div([
                html.Div(id='file-summary-header', style={ 'whiteSpace': 'normal'}),
                html.Div(
                    id='file-summary-details',
                    style={
                        'padding': '10px',
                        'border': '1px solid black',
                        'margin-bottom': '20px',
                        'width': '100%',
                        'whiteSpace': 'normal',
                        'display': 'flex',
                        'flexDirection': 'row'
                    }
                ),
            ], style={'display': 'flex', 'flexDirection': 'column', 'width': '100%'}),
            dcc.Loading(
                html.Div(id='attribute-content', style={'width': '100%', 'whiteSpace': 'normal'})
            )
        ], width=10)
    ], style={'minHeight': '100vh', 'display': 'flex', 'flexDirection': 'row'}),
    dbc.Row([
        dbc.Col(
            dbc.Button("Back to Index", id='back-to-index-btn', color='secondary', href='/'),
            width={'size': 2, 'offset': 5},
            style={'margin-top': '20px'}
        ),
        dbc.Col(
            dbc.Button("View Details", id='view-details-btn', color='primary', disabled=True, href='/details'),
            style={'display':'none'}
        )]
    )
], fluid=True)  # Set the container to fluid to cover full width



# Update index layout based on the URL
@app.callback(
    Output('page-content', 'children'),
    Input('url', 'pathname'))
def display_page(pathname):
    if pathname == '/details':
        return detail_page
    else:
        return index_page

# Define callback to update table based on filters
@app.callback(
    Output('data_table', 'data'),
    Input('sort_by', 'value'),
    Input('file_type_filter', 'value')
)
def update_table(sort_by, file_type_filter):
    filtered_df = df_summary.copy()
    if file_type_filter:
        # Filter based on selected extensions
        filtered_df = filtered_df[filtered_df['extension'].isin(file_type_filter)]

    if sort_by:
        filtered_df = filtered_df.sort_values(by=sort_by)
        df_summary = filtered_df

    return filtered_df.to_dict('records')

# Enable/disable view details button based on row selection
@app.callback(
    Output('view-details-btn', 'disabled'),
    Input('data_table', 'selected_rows')
)
def enable_view_details_btn(selected_rows):
    return not bool(selected_rows)

@app.callback(
    [Output('url', 'pathname'),
     Output('url', 'search')],
    [Input('view-details-btn', 'n_clicks'),
     Input('back-to-index-btn', 'n_clicks')],
    [State('data_table', 'selected_rows')]
)
def handle_navigation(view_details_clicks, back_to_index_clicks, selected_rows):
    # Handle navigation to details page
    print(selected_rows)
    if view_details_clicks and selected_rows:
        selected_index = selected_rows[0]  # Get the index of the selected row
        return '/details', f'?index={selected_index}'  # Pass the index as a query parameter

    # Handle navigation back to index page
    if back_to_index_clicks:
        return '/', ''  # Clear query parameters

    # Default return when no action is taken
    return dash.no_update, dash.no_update

# @app.callback(
#     Output('url', 'pathname'),
#     Output('url', 'search'),
#     Input('view-details-btn', 'n_clicks'),
#     State('data_table', 'selected_rows')
# )
# def view_details(n_clicks, selected_rows):
#     if n_clicks and selected_rows:
#         selected_index = selected_rows[0]  # Get the index of the selected row
#         return '/details', f'?index={selected_index}'  # Pass the index as a query parameter
#     return dash.no_update, dash.no_update

# @app.callback(
#     Output('url', 'pathname'),
#     Input('back-to-index-btn', 'n_clicks')
# )
# def go_back(n_clicks):
#     if n_clicks > 0:
#         return '/'
#     return dash.no_update


@app.callback(
    [Output('file-summary-header', 'children'),
     Output('file-summary-details', 'children'),
     Output('attribute-nav', 'children')],
    [Input('url', 'search')]
)
def display_dataset_details(search):
    # Parse the query string
    params = urllib.parse.parse_qs(urllib.parse.urlparse(search).query)
    index = int(params.get('index', [''])[0])
    if index >= 0 and index < len(df_summary):
        selected_file = df_summary.iloc[index]
        print(selected_file)
        df = load_single_file(os.path.join(root_directory_path, selected_file['name']))
#         print(df)
        
        if not (DetectHeader(df)):
            if df is not None:
                df.columns = [f'Column {i+1}' for i in range(df.shape[1])]
        
                file_name = os.path.basename(selected_file['name'])
                size_mb = df.memory_usage(deep=True).sum() / (1024 * 1024)
                attributes = df.columns.tolist()
                file_format = os.path.splitext(file_name)[1].lower() 

                filename =  html.H6(f"File Summary: {file_name}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),

                nav_links = [dbc.NavLink(col, href=f"/details?index={index}&attr={col}", id=f"sidebar-{col}") for col in attributes]

                summary_details = [
                    html.Div(children = [
                    html.Div(f"Name: {file_name}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                    html.Div(f"Size: {size_mb:.2f} MB", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                    html.Div(f"File Format: {file_format}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                    html.Div(f"Total Completeness: {df_summary.iloc[index].completeness_score:.2f}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                    html.Div(f"Total Uniqueness: {df_summary.iloc[index].uniqueness_score:.2f}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                    ], style = {'display' : 'flex', 'flexDirection': 'row'})]

                return filename, html.Div(summary_details), nav_links

            return '', [], []


# Update the detailed view based on selected attribute
@app.callback(Output('attribute-content', 'children'),[Input('url', 'search')]
)
def update_attribute_content(search):
    # Parse the query string
    params = urllib.parse.parse_qs(urllib.parse.urlparse(search).query)
    index = int(params.get('index', [''])[0])
    attribute = params.get('attr', [''])[0]
    if attribute is not None:
        attribute = unquote(attribute)
    print(df_summary.iloc[index])
    if index >= 0 and index < len(df_summary):
        selected_file = df_summary.iloc[index]
        df = load_single_file(os.path.join(root_directory_path, selected_file['name']))

        if  not (DetectHeader(df)):
            if df is not None:
                df.columns = [f'Column {i+1}' for i in range(df.shape[1])]
                if attribute is None:
                    attribute = df.columns[0]  


                if attribute not in df.columns:
                    return html.Div("Invalid attribute attribute")

                attribute_data = df[attribute]
                outliers = pd.DataFrame()
                profiler = DataProfiler(df, attribute)
                x_min, x_max , y_max = 0, 0, 0
                max_categories = 0
            
                print("column 1 is ",attribute)

                if pd.api.types.is_numeric_dtype(attribute_data):
                    # Plot numeric data
                    line_fig = px.histogram(attribute_data, title=f'Distribution of {attribute}')
                    graph_fig = line_fig
        #             print("debug")
                    # Detect and display outliers
        #             print(attribute)
                    outliers = profiler.detect_outliers_iqr(df, attribute)
                    # Add sliders for histogram
                    x_min, x_max = attribute_data.min(), attribute_data.max()
                    y_max = attribute_data.value_counts().max()
                    # **Calculate skew and kurtosis**
                    column_skew = skew(attribute_data.dropna())  # **Skew**
                    column_kurtosis = kurtosis(attribute_data.dropna())  # **Kurtosis**

                else:
                    # Plot categorical data
                    value_counts = attribute_data.value_counts()
                    pie = px.pie(values=value_counts.values, names=value_counts.index, title=f'Distribution of {attribute}')
                    graph_fig = pie
        #             print("debug")
                    # Detect and display outliers
                    outliers = profiler.detect_outliers_frequency(df, attribute)

                    # Slider for top N categories with label
                    max_categories = len(value_counts)

        #         print(outliers)

                sliders = html.Div([
                    html.Div(id = "numericSlider",children = [ 
                        html.Label("X-Axis Range:"),
                        dcc.RangeSlider(
                            id='x-range-slider',
                            min=x_min,
                            max=x_max,
                            step=(x_max - x_min) / 100,  # Divide into 100 steps
                            value=[x_min, x_max],
                            marks={i: str(i) for i in np.linspace(x_min, x_max, num=5)}
                        ),
                        html.Label("Y-Axis Range:"),
                        dcc.RangeSlider(
                            id='y-range-slider',
                            min=0,
                            max=y_max,
                            step=y_max / 100,  # Divide into 100 steps
                            value=[0, y_max],
                            marks={i: str(int(i)) for i in np.linspace(0, y_max, num=5)}
                        )
                    ]),
                    html.Div(id = "categoricalSlider",children = [
                        html.Label("Top N Categories:"),
                        dcc.Slider(
                            id='top-n-slider',
                            min=1,
                            max=max_categories,
                            step=1,
                            value=min(5, max_categories),  # Default to top 5 categories
                            marks={i: str(i) for i in range(1, max_categories + 1)}
                        )
                    ])
                ])

                column_profile = {
                    'Completeness': profiler.completeness_score(),
                    'ApproxCountDistinct': profiler.uniqueness_score(),
                    'Mean': df[attribute].mean() if pd.api.types.is_numeric_dtype(attribute_data) else 'N/A',
                    'Minimum': df[attribute].min() if pd.api.types.is_numeric_dtype(attribute_data) else 'N/A',
                    'Maximum': df[attribute].max() if pd.api.types.is_numeric_dtype(attribute_data) else 'N/A',
                    'StandardDeviation': df[attribute].std() if pd.api.types.is_numeric_dtype(attribute_data) else 'N/A',
                    'Sum': df[attribute].sum() if pd.api.types.is_numeric_dtype(attribute_data) else 'N/A',
                    'Count': df[attribute].count(),
                    'IsIndex': profiler.isIndex(),
                    'MeanMedianImbalance' : profiler.mean_median_difference()}

                completeness_table = dbc.Table.from_dataframe(pd.DataFrame({
                    'Metric': ['Completeness Score', 'Uniqueness Score', 'No of Outliers'],
                    'Value': [column_profile['Completeness'], column_profile['ApproxCountDistinct'], outliers.shape[0]]
                }), striped=True, bordered=True, hover=True)

                dataDesc = dbc.Table.from_dataframe(pd.DataFrame({
                    "Metric": ["Mean", "Minimum", "Maximum", "Standard Deviation", "Sum", "Count", "Mean/Median Imbalance"],
                    "Value": [column_profile['Mean'], column_profile['Minimum'], column_profile['Maximum'], column_profile['StandardDeviation'], column_profile['Sum'], column_profile['Count'], column_profile['MeanMedianImbalance']]
                }), striped=True, bordered=True, hover=True)

                 # Filter out the outliers for the selected attribute
        #         outliers_for_attribute = outliers[outliers['Attribute'] == attribute]

                # Prepare outliers table
                outliers_table = dbc.Table.from_dataframe(outliers, striped=True, bordered=True, hover=True)

                # Display outliers if any
                outlier_info = html.Div()
                if not outliers.empty:
                    outlier_info = html.Div([
                        html.H6(f"Potential Erroneous values in '{attribute}' : ", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                        outliers_table
                    ])

                 # **Display skew and kurtosis if the data is numeric**
                skew_kurtosis_info = html.Div()
                if pd.api.types.is_numeric_dtype(attribute_data):
                    skew_kurtosis_info = html.Div([
                        html.H6(f"Skew: {column_skew:.2f}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '10px'}),
                        html.H6(f"Kurtosis: {column_kurtosis:.2f}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'})
                    ])

                return html.Div([
                    html.H6(f"Details for Attribute: '{attribute}'", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                    html.H6(f"Is '{attribute}' a Potential Index Column: {'Yes' if column_profile['IsIndex'] else 'No'}", style={'padding': '10px', 'border': '1px solid black', 'margin-bottom': '20px'}),
                    dbc.Row([
                        dbc.Col(html.Div([
                            html.H6(f"Distribution of {attribute}"),
                            dcc.Graph(id='graph', figure=graph_fig,
                            config={
                                    'scrollZoom': True,    # Enables zooming with the scroll wheel
                                    'displayModeBar': True,  # Displays the mode bar (tool bar)
                                    'displaylogo': False,  # Hides the Plotly logo
                                    'modeBarButtonsToRemove': ['select2d', 'lasso2d'],  # Customizing mode bar buttons
                                    'staticPlot': False  # Allows full interactivity
                                })
                        ]), width=12)
                    ]),
                    sliders,
                    dbc.Row([
                        dbc.Col(html.Div([
                            html.H6("Data Description"),
                            dataDesc,
                        ]), width=6),
                        dbc.Col([
                            skew_kurtosis_info
                        ], width=6)
                    ]),
                    html.H6("Completeness Table"),
                    completeness_table,
                    outlier_info
                ])

    

@app.callback([Output('numericSlider', 'style'),Output('categoricalSlider', 'style')],[Input('url', 'search')]
)
def update_sliders_visibility(search):
#     print("visibility")
    params = urllib.parse.parse_qs(urllib.parse.urlparse(search).query)
    index = int(params.get('index', [''])[0])
    attribute = params.get('attr', [''])[0]
    if attribute is not None:
        attribute = unquote(attribute)

    if index >= 0 and index < len(df_summary):
        selected_file = df_summary.iloc[index]
        df = load_single_file(os.path.join(root_directory_path, selected_file['name']))
        
        if  not (DetectHeader(df)):
            if df is not None:
                df.columns = [f'Column {i+1}' for i in range(df.shape[1])]
                
                if attribute is None : 
                    attribute = df.columns[0]
        
                if attribute in df.columns:
                    attribute_data = df[attribute]
        #             print(attribute_data)
                    if pd.api.types.is_numeric_dtype(attribute_data):
        #                 print("entered numeric in visibility " )
                        return [{'display': 'block'}, {'display': 'none'}]
                    else:
                        return [{'display': 'none'},{'display': 'block'}]

    return [{'display': 'none'}, {'display': 'none'}]


    
    
@app.callback(
    Output('graph', 'figure'),
    [Input('x-range-slider', 'value'),
     Input('y-range-slider', 'value'),
    Input('top-n-slider' , 'value')],
    [State('url', 'search')]
)
def update_graph(x_range, y_range, top_n, search):
    ctx = callback_context
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0] if ctx.triggered else 'None'

    params = urllib.parse.parse_qs(urllib.parse.urlparse(search).query)
    index = int(params.get('index', [''])[0])
    attribute = params.get('attr', [''])[0]
    if attribute is not None:
        attribute = unquote(attribute)

    if index >= 0 and index < len(df_summary):
        selected_file = df_summary.iloc[index]
        df = load_single_file(os.path.join(root_directory_path, selected_file['name']))
        
        if  not (DetectHeader(df)):
            if df is not None:
                df.columns = [f'Column {i+1}' for i in range(df.shape[1])]
                if attribute is None : 
                    attribute = df.columns[0]
        
                if attribute not in df.columns:
                    return dash.no_update

                attribute_data = df[attribute]

                if pd.api.types.is_numeric_dtype(attribute_data):
                    if 'x-range-slider' in triggered_id or 'y-range-slider' in triggered_id:
                        filtered_data = attribute_data[(attribute_data >= x_range[0]) & (attribute_data <= x_range[1])]
                        hist_fig = px.histogram(filtered_data, title=f'Distribution of {attribute}')
                        hist_fig.update_layout(yaxis_range=y_range)
                        return hist_fig

                else:
                # Handle categorical data
                    # Limit data to top_n categories and include 'Others'
                    value_counts = attribute_data.value_counts()
                    top_categories = value_counts.head(top_n)
                    other_count = value_counts.iloc[top_n:].sum()

                    if other_count > 0:
                        top_categories = top_categories.append(pd.Series([other_count], index=['Others']))

        #             print("top categories - ", top_categories)

                    pie_fig = px.pie(values=top_categories.values, names=top_categories.index, title=f'Distribution of {attribute}')
                    return pie_fig

    return dash.no_update

     

if __name__ == '__main__':
    app.run_server(debug=True)
